In [1]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame

In [2]:

db = pd.read_csv('Data.csv')

In [3]:
db.loc[(db['Response']==0) & (db['BMI']==0.0), "BMI"]=30.3042

db.loc[(db['Response']==1) & (db['BMI']==0.0), "BMI"]=35.142537

db.loc[(db['Response']==0) & (db['Insulin']==0.0), "Insulin"]=68.792

db.loc[(db['Response']==1) & (db['Insulin']==0.0), "Insulin"]=100.335821

db.loc[(db['Response'] == 0) & (db['Pglucose'] == 0.0), "Pglucose"]=109.98
db.loc[(db['Response'] == 1) & (db['Pglucose'] == 0.0), "Pglucose"] =141.257463 

db.loc[(db['Response']==0) & (db['Skin']==0.0), "Skin"]=19.664
db.loc[(db['Response']==1) & (db['Skin']==0.0), "Skin"]=22.164

db.loc[(db['Response']==0) & (db['DBP']==0.0), "DBP"]=68.184
db.loc[(db['Response']==1) & (db['DBP']==0.0), "DBP"]=70.824627

In [4]:
data = DataFrame(np.zeros(768*24).reshape(768,24), columns = ['preg1','preg2','preg3','pg1','pg2','pg3','dbp1','dbp2','dbp3','skin1','skin2','skin3',
                                                             'insulin1','insulin2','insulin3','bmi1','bmi2','bmi3','dpf1','dpf2','dpf3','age1','age2','age3'])
data['Response'] = db['Response']
#data.head()

In [5]:
for i in range(0,768):
    if db.ix[i,'Preg']<=1:
        data.loc[i,'preg1'] = 1
    elif db.ix[i,'Preg']<=3:
        data.loc[i,'preg2'] = 1
    elif db.ix[i,'Preg']<=6:
        data.loc[i,'preg3'] = 1
    
    if db.ix[i,'Pglucose']<=99.75:
        data.loc[i,'pg1'] = 1
    elif db.ix[i,'Pglucose']<=117:
        data.loc[i,'pg2'] = 1
    elif db.ix[i,'Pglucose']<=141:
        data.loc[i,'pg3'] = 1
    
    if db.ix[i,'DBP']<=64:
        data.loc[i,'dbp1'] = 1
    elif db.ix[i,'DBP']<=72:
        data.loc[i,'dbp2'] = 1
    elif db.ix[i,'DBP']<=80:
        data.loc[i,'dbp3'] = 1
    
    if db.ix[i,'Skin']<=19.664:
        data.loc[i,'skin1'] = 1
    elif db.ix[i,'Skin']<=23:
        data.loc[i,'skin2'] = 1
    elif db.ix[i,'Skin']<=32:
        data.loc[i,'skin3'] = 1
    
    if db.ix[i,'Insulin']<=68.792:
        data.loc[i,'insulin1'] = 1
    elif db.ix[i,'Insulin']<=100:
        data.loc[i,'insulin2'] = 1
    elif db.ix[i,'Insulin']<=127.25:
        data.loc[i,'insulin3'] = 1
    
    if db.ix[i,'BMI']<=27:
        data.loc[i,'bmi1'] = 1
    elif db.ix[i,'BMI']<=32:
        data.loc[i,'bmi2'] = 1
    elif db.ix[i,'BMI']<=36:
        data.loc[i,'bmi3'] = 1
    
    if db.ix[i,'Dpfunc']<=0.24375:
        data.loc[i,'dpf1'] = 1
    elif db.ix[i,'Dpfunc']<=0.3725:
        data.loc[i,'dpf2'] = 1
    elif db.ix[i,'Dpfunc']<=0.62650:
        data.loc[i,'dpf3'] = 1
    
    if db.ix[i, 'Age']<=24:
        data.loc[i,'age1'] = 1
    elif db.ix[i, 'Age'] <=29:
        data.loc[i,'age2']=1
    elif db.ix[i,'Age']<=41:
        data.loc[i,'age3']=1

In [6]:
import random

In [7]:
data = data.iloc[np.random.permutation(len(data)-1)]
train_indices = random.sample(range(0,768),576)
test_indices = list()
for i in range(0,len(data)):
    if i in train_indices:
        continue
    else:
        test_indices.append(i)


In [8]:
train_data = data.loc[train_indices]
test_data  = data.loc[test_indices]
train_data.reset_index(drop=True,inplace=True)
test_data.reset_index(drop=True,inplace=True)

In [9]:
cols = list(data)
cols.pop(24)
cols.sort()

In [10]:
for col in cols:
    test_data[col] = test_data[col].fillna(0).astype(int)
    train_data[col] = train_data[col].fillna(0).astype(int)
test_data['Response'] = test_data['Response'].fillna(0).astype(int)
train_data['Response']=train_data['Response'].fillna(0).astype(int)

In [11]:
params_1 = list()
params_0 = list()
for col in cols:
    params_1.append(train_data.groupby(train_data['Response'])[col].describe()[1][1])
    params_0.append(train_data.groupby(train_data['Response'])[col].describe()[0][1])

In [12]:
params_0
params_1

[0.10000000000000001,
 0.20526315789473684,
 0.35789473684210527,
 0.057894736842105263,
 0.25789473684210529,
 0.28947368421052633,
 0.16315789473684211,
 0.28421052631578947,
 0.26842105263157895,
 0.17894736842105263,
 0.25263157894736843,
 0.22105263157894736,
 0.031578947368421054,
 0.04736842105263158,
 0.59473684210526312,
 0.052631578947368418,
 0.18947368421052632,
 0.26315789473684209,
 0.22105263157894736,
 0.15789473684210525,
 0.25789473684210529,
 0.042105263157894736,
 0.40000000000000002,
 0.23684210526315788]

In [13]:
phi_1 = (train_data.groupby(train_data['Response'])['age1'].describe()[1][0])/len(train_data)
phi_0 = 1 - phi_1

In [14]:
phi_1
phi_0

0.67013888888888884

In [15]:
backup = test_data
resp = test_data['Response']
test_data = test_data.drop('Response',axis=1).sort_index(axis=1)
test_data['Response'] = resp

In [17]:
#Python doesn't have an in-buit product function so we have to define a user product function

import operator
def prod(iterable):
    return reduce(operator.mul, iterable, 1)

In [18]:
predict=list()
for i in range(0,len(test_data)):
    test_p1=[sum(x) for x in zip(test_data.ix[i],params_1)]
    test_p0=[sum(x) for x in zip(test_data.ix[i],params_0)]
    
    for j in range(0, len(test_p1)):
        if test_p1[j]>1:
            test_p1[j]-=1
        elif test_p1[j]<1:
            test_p1[j] = 1-test_p1[j]
        
        if test_p0[j]>1:
            test_p0[j]-=1
        elif test_p0[j]<1:
            test_p0[j]=1-test_p0[j]
            
    p1 = (prod(test_p1)*phi_1)/((prod(test_p1)*phi_1)+(prod(test_p0)*phi_0))
    p0 = (prod(test_p0)*phi_0)/((prod(test_p1)*phi_1)+(prod(test_p0)*phi_0))
    
    if p1 > p0:
        predict.append(1)
    else:
        predict.append(0)

In [19]:
#Checking performance of classifier algorithm
count = 0.0
for i in range(0,len(test_data)):
    if test_data.ix[i,'Response']==predict[i]:
        count+=1
count
perform = count/len(test_data)
perform
    

0.84375

In [20]:
#IMPLENTATION OF NAIVE BAYES USING SCI-KIT LEARN
import sklearn
from sklearn.naive_bayes import BernoulliNB
gnb = BernoulliNB()

In [21]:
train = train_data.drop('Response',axis=1)
test = test_data.drop('Response',axis=1)
pred = gnb.fit(train,  train_data['Response']).predict(test)

In [22]:
#Checking performance of classifier algorithm
count = 0.0
for i in range(0,len(test_data)):
    if test_data.ix[i,'Response']==pred[i]:
        count+=1
count
perform = count/len(test_data)
perform

0.8072916666666666